<h2>개인 구글 드라이브와 colab 연동</h2>

In [18]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [19]:
import numpy as np

file_path = "/gdrive/My Drive/Colab Notebooks/svm/SMSSpamCollection.dat"

# 파일 읽기
x_data, y_data = [], []
x_data_com = []
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()
  
lines = lines[:100]  
  
for line in lines:
  # 탭으로 분리 \t
  line = line.strip().split('\t')
  # line[0] 정답 label
  sentence, label = line[1], line[0]
  x_data.append(sentence)
  x_data_com.append(sentence)
  y_data.append(label)
  
print("x_data의 개수 : " + str(len(x_data)))
print("y_data의 개수 : " + str(len(y_data)))

x_data의 개수 : 100
y_data의 개수 : 100


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# 불용어 list
filter =  nltk.corpus.stopwords.words('english')

# 불용어 제거를 위해 문장을 공백 기준으로 구분해 list를 만들어줍니다.
for idx in range(len(x_data)):
  x_data[idx] = x_data[idx].split(' ')

# 불용어 제거 과정
# 시간 복잡도가 n^2, 데이터가 많으면 성능이 안 나올 것 같습니다.
for data in x_data:
  # 한 문장이 포함하는 불용어를 임시적으로 추가할 list 입니다.
  tmp = []
  # 제거가 잘 되었는지 확인을 위한 print
  print(' '.join(data))
  for idx in range(len(data)):
    # 불용어 list의 요소들은 모두 소문자 이므로 lower()로 비교해 줍니다.
    if(data[idx].lower() in filter): 
      tmp.append(data[idx])
  # 한 문장내의 불용어를 제거해줍니다.
  for item in tmp:
    data.remove(item)
  # 불용어가 제거된 문장의 공백을 기준으로 split 했던 문장들을 다시 join 해줍니다.
  data = ' '.join(data)
  # 제거가 잘 되었는지 확인을 위한 print
  print(data)
  print()

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Go jurong point, crazy.. Available bugis n great world la e buffet... Cine got amore wat...

Ok lar... Joking wif u oni...
Ok lar... Joking wif u oni...

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
Free entry 2 wkly comp win FA Cup final tkts 21st May 2005. Text FA 87121 receive entry question(std txt rate)T&C's apply 08452810075over18's

U dun say so early hor... U c already then say...
U dun say early hor... U c already say...

Nah I don't think he goes to usf, he lives around here though
Nah think goes usf, lives around though

FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, ￡1.50 to rcv
FreeMsg Hey darling 3 week's word back! I'd like fun still? Tb ok! XxX std chgs send, ￡1.50

In [22]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

# spam, ham 라벨을 대응하는 index로 치환하기위한 딕셔너리
label2index_dict = {'spam':0, 'ham':1}

# indexing 한 데이터를 넣을 리스트 선언
indexing_x_data, indexing_y_data = [], []

for label in y_data:
  indexing_y_data.append(label2index_dict[label])

# x_data를 사용하여 딕셔너리 생성
tokenizer.fit_on_texts(x_data)                   

# x_data에 있는 각 문장의 단어들을 대응하는 index로 치환하고 그 결과값을 indexing_x_data에 저장
indexing_x_data = tokenizer.texts_to_sequences(x_data)    

print("x_data indexing 하기 전 : " + str(x_data[0]))
print("x_data indexing 하기 후 : " + str(indexing_x_data[0]))
print("y_data indexing 하기 전 : " + str(y_data[0]))
print("y_data indexing 하기 후 : " + str(indexing_y_data[0]))

x_data indexing 하기 전 : ['Go', 'jurong', 'point,', 'crazy..', 'Available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'got', 'amore', 'wat...']
x_data indexing 하기 후 : [7, 145, 146, 147, 148, 149, 36, 52, 150, 151, 53, 152, 153, 19, 154, 155]
y_data indexing 하기 전 : ham
y_data indexing 하기 후 : 1


In [23]:
from sklearn.svm import SVC

# 문장의 길이를 max_length으로 맞춰 변환
max_length = 60
for index in range(len(indexing_x_data)):
  length = len(indexing_x_data[index])
  
  if(length > max_length):
    indexing_x_data[index] = indexing_x_data[index][:max_length]
  elif(length < max_length):
    indexing_x_data[index] = indexing_x_data[index] + [0]*(max_length-length)
    
    
# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
number_of_train = int(len(indexing_x_data)*0.9)

train_x = indexing_x_data[:number_of_train]
train_y = indexing_y_data[:number_of_train]
test_x = indexing_x_data[number_of_train:]
test_y = indexing_y_data[number_of_train:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

svm = SVC(kernel='linear', C=1e10)
svm.fit(train_x, train_y)

train_x의 개수 : 90
train_y의 개수 : 90
test_x의 개수 : 10
test_y의 개수 : 10


SVC(C=10000000000.0, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [24]:
predict = svm.predict(test_x)

correct_count = 0
for index in range(len(predict)):
  if(test_y[index] == predict[index]):
    correct_count += 1
    
accuracy = 100.0*correct_count/len(test_y)


print("Accuracy: " + str(accuracy))

index2label = {0:"spam", 1:"ham"}

test_x_word = tokenizer.sequences_to_texts(test_x)

for index in range(len(test_x_word)):
  print()
  print("문장 : ", test_x_word[index])
  print("정답 : ", index2label[test_y[index]])
  print("모델 출력 : ", index2label[predict[index]])

Accuracy: 60.0

문장 :  yeah do! don‘t stand close tho- you‘ll catch something!
정답 :  ham
모델 출력 :  ham

문장 :  sorry pain. ok meet another night? spent late afternoon casualty means done stuff42moro includes time sheets that. sorry.
정답 :  ham
모델 출력 :  ham

문장 :  smile pleasure smile pain smile trouble pours like rain smile sum1 hurts u smile becoz someone still loves see u smiling!!
정답 :  ham
모델 출력 :  spam

문장 :  please call customer service representative 0800 169 6031 10am-9pm guaranteed ￡1000 cash ￡5000 prize!
정답 :  spam
모델 출력 :  ham

문장 :  havent planning buy later. check already lido got 530 show e afternoon. u finish work already?
정답 :  ham
모델 출력 :  spam

문장 :  free ringtone waiting collected. simply text password "mix" 85069 verify. get usher britney. fml, po box 5249, mk17 92h. 450ppw 16
정답 :  spam
모델 출력 :  spam

문장 :  watching telugu movie..wat abt u?
정답 :  ham
모델 출력 :  ham

문장 :  see. finish loads loans pay
정답 :  ham
모델 출력 :  ham

문장 :  hi. wk ok - hols now! yes bit run. forgot 